
# Coupes verticales de tempêtes dans les réanalyses ERA5

Calepin à utiliser pour le tracé de coupes verticales de tempêtes ERA5 pour alimenter le site tempêtes : http://tempetes.canevas.enm.meteo.fr/tempete.py.

- Pour chaque tempête, un fichier de tracking "tempete.txt" obtenu grâce au calepin "ERA5_storms_tracking_auto.ipynb" doit être présent dans le répertoire 'txt/tempete/'
- Les données netcdf de Pmer "msl.nc" ont été téléchargées sur Copernicus au pas de temps horaire et sur domaine limité (-100W-50E, 0-90N) : https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=form

In [ ]:
import os
import xarray as xr
import numpy as np

from cartopy import config
from cartopy.util import add_cyclic_point
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid

from scipy.ndimage import gaussian_filter

import metpy
import metpy.calc as mpcalc
from metpy.interpolate import cross_section

from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def lonflip(da):
    lon_name = 'longitude'
    da['_longitude_adjusted'] = xr.where(
        da[lon_name] > 180,
        da[lon_name] - 360,
        da[lon_name])
    da = (
        da
        .swap_dims({lon_name: '_longitude_adjusted'})
        .sel(**{'_longitude_adjusted': sorted(da._longitude_adjusted)})
        .drop(lon_name))
    da = da.rename({'_longitude_adjusted': lon_name})
    return da

In [ ]:
#storms=['Nov1982', 'Oct1987', 'Herta', 'Viviane', 'Braer', 'Lothar', 'Martin', 'Klaus', 'Xynthia',
#        'Joachim', 'Zeus', 'Ophelia', 'Eleanor', 'Zorbas', 'Lorenzo', 'Alex', 'Ciaran']

storm='Ciaran' # single storm

dir_data="E:/BIG_DATA/ERA5_storms/data/"
dir_diag="E:/BIG_DATA/ERA5_storms/diagnostics/"
dir_figs='./figs/'
dir_txt='./txt/'
dir_anim='./anim/'

if not os.path.exists(dir_figs+storm):
    os.mkdir(dir_figs+storm)
if not os.path.exists(dir_anim+storm):
    os.mkdir(dir_anim+storm)
if not os.path.exists(dir_txt+storm+'.txt'):
    print('Missing tracking file for storm '+storm)
if not os.path.exists('./figs/'+storm+'/Cross_section/'):
    os.mkdir('./figs/'+storm+'/Cross_section/')      

In [ ]:
#--  Open storm track file
liste_time,liste_lon,liste_lat,liste_pres=np.loadtxt(dir_txt+storm+'.txt',skiprows=0, dtype='U13,float,float,int',unpack=True)
date1=str(liste_time[0])
date2=str(liste_time[-1])

In [ ]:
latS=30
latN=70
lonW=-100
lonE=50

if storm=='Lorenzo':
    latS=25
    latN=70
    lonW=-100
    lonE=50

In [ ]:
fmsl    = xr.open_mfdataset(dir_data+storm+"/msl*.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS))
fu    = xr.open_dataset(dir_data+storm+"/u.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS))
fv    = xr.open_dataset(dir_data+storm+"/v.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS))
fpv    = xr.open_dataset(dir_data+storm+"/pv.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS))
fw    = xr.open_dataset(dir_data+storm+"/w.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS))
fth    = xr.open_dataset(dir_diag+storm+"/thetae.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS))

In [ ]:
fpv['u_wind'] = fu['u']
fpv['v_wind'] = fv['v']
fpv['w_wind'] = fw['w']
fpv['thetae'] = fth['thetae']

print(fpv)

In [ ]:
#--  Parse datasets
mslp=fmsl.metpy.parse_cf().squeeze()
data=fpv.metpy.parse_cf().squeeze()
mslp = lonflip(mslp)
data = lonflip(data)
time  = data.time.values
print(mslp)
print(data)

In [ ]:
mslp_levels = np.arange(900,1072,2)
pv_levels = np.arange(0,4.2,0.2)
th_levels=np.arange(-20,75,5)

## Coupes zonales

In [ ]:
for i in tqdm(range(len(time))):

	#print(str(time[i])[0:13])

	mslp2=mslp.sel(time=time[i])
	data2=data.sel(time=time[i])

#--  Cross sections with Metpy

# zonal
	start_z = (liste_lat[i], liste_lon[i]-10)
	end_z = (liste_lat[i], liste_lon[i]+10)
    
	#print('Building cross sections ...')
	cross_z=metpy.interpolate.cross_section(data2, start_z, end_z, steps=100, interp_type='linear').set_coords(('latitude', 'longitude'))
	cross_z['t_wind'], cross_z['n_wind'] = mpcalc.cross_section_components(cross_z['u_wind'],cross_z['v_wind'])

	#print(cross_z)

#--  plot zonal cross sections

	fig = plt.figure(figsize=(16., 9.))

	fig.suptitle('Storm '+storm+' - zonal cross section  : Potential vorticity '+r"$\theta_e$ "+'and winds',fontsize=16)
	ax = plt.axes()
	ax.set_title(str(time[i])[0:13],loc='center',fontsize=14)
	ax.set_ylim(cross_z['level'].max(), cross_z['level'].min()) #met la pression min au sommet du graphe 
	ax.set_ylabel('Pressure (hPa)')
	ax.set_xlabel('Longitude')
	c = ax.contourf(cross_z['longitude'], cross_z['level'], cross_z['pv']* 1e06, levels=pv_levels, cmap='YlOrRd',extend='both')	
	cb = fig.colorbar(c)
	cb.set_label('Potential vorticity', size='large')
	c1 = ax.contour(cross_z['longitude'], cross_z['level'], cross_z['pv']* 1e06, levels = pv_levels, colors='grey', linewidths=0.0001)
	c2 = ax.contour(cross_z['longitude'], cross_z['level'], cross_z['pv']* 1e06, levels=[1.5], colors='black', linewidths=2)
	c3 = ax.contour(cross_z['longitude'], cross_z['level'], cross_z['thetae'], levels = th_levels, colors='brown', linewidths=1)
	ax.clabel(c3,fmt='%2.1i',fontsize=10)
	wind = ax.barbs(cross_z['longitude'][5::6], cross_z['level'], cross_z['u_wind'][:,5::6], cross_z['v_wind'][:,5::6], color='navy')

# Plot small stamp map
	data_crs = mslp['msl'].metpy.cartopy_crs
	ax_inset = fig.add_axes([0.125, 0.695, 0.2, 0.2], projection=data_crs)
	#ax_inset.set_title('Coupe verticale et Pmer')
	ax_inset.coastlines()
	ax_inset.gridlines()
	bounds = [(lonW, lonE, latS, latN)]
	ax_inset.set_extent(*bounds, crs=data_crs)
	# Plot MSLP
	ax_inset.contour(mslp['longitude'], mslp['latitude'], mslp['msl'][i,:,:]/100, levels=mslp_levels, colors='blue', linewidths=0.5, transform=ccrs.PlateCarree())
	#ax_inset.plot(cross['longitude'], cross['latitude'], transform=ccrs.PlateCarree(), color='black')
	# Plot the path of the cross section
	endpoints = data_crs.transform_points(ccrs.Geodetic(),*np.vstack([start_z, end_z]).transpose()[::-1])
	ax_inset.scatter(endpoints[:, 0], endpoints[:, 1], c='k', zorder=2)
	ax_inset.plot(cross_z['longitude'], cross_z['latitude'], c='k', zorder=2)

	#plt.show()	

	figname='./figs/'+storm+'/Cross_section/PV_winds_zonal_'+str(time[i])[0:13]
	fig.savefig(figname+'.png',bbox_inches='tight')

	plt.close()

## Coupes méridiennes

In [ ]:
for i in tqdm(range(len(time))):

	#print(str(time[i])[0:13])

	mslp2=mslp.sel(time=time[i])
	data2=data.sel(time=time[i])

#--  Cross sections with Metpy

# meridional
	start_m = (liste_lat[i]-10, liste_lon[i])
	end_m = (liste_lat[i]+10, liste_lon[i])
    
	#print('Building cross sections ...')

	cross_m=metpy.interpolate.cross_section(data2, start_m, end_m, steps=100, interp_type='linear').set_coords(('latitude', 'longitude'))		
	cross_m['t_wind'], cross_m['n_wind'] = mpcalc.cross_section_components(cross_m['u_wind'],cross_m['v_wind'])

	#print(cross_m)

#--  plot meridional cross section

	fig = plt.figure(figsize=(16., 9.))

	fig.suptitle('Storm '+storm+' - meridional cross section  : Potential vorticity '+r"$\theta_e$ "+'and winds',fontsize=16)
	ax = plt.axes()
	ax.set_title(str(time[i])[0:13],loc='center',fontsize=14)
	ax.set_ylim(cross_m['level'].max(), cross_m['level'].min()) #met la pression min au sommet du graphe 
	ax.set_ylabel('Pressure (hPa)')
	ax.set_xlabel('Latitude')
	c = ax.contourf(cross_m['latitude'], cross_m['level'], cross_m['pv']* 1e06, levels=pv_levels, cmap='YlOrRd',extend='both')	
	cb = fig.colorbar(c)
	cb.set_label('Potential vorticity', size='large')
	c1 = ax.contour(cross_m['latitude'], cross_m['level'], cross_m['pv']* 1e06, levels = pv_levels, colors='grey', linewidths=0.0001)
	c2 = ax.contour(cross_m['latitude'], cross_m['level'], cross_m['pv']* 1e06, levels=[1.5], colors='k', linewidths=2)
	c3 = ax.contour(cross_m['latitude'], cross_z['level'], cross_z['thetae'], levels = th_levels, colors='brown', linewidths=1)
	ax.clabel(c3,fmt='%2.1i',fontsize=10)
	wind = ax.barbs(cross_m['latitude'][5::6], cross_m['level'], cross_m['u_wind'][:,5::6], cross_m['v_wind'][:,5::6], color='navy')

	# Define the CRS and inset axes
	data_crs = mslp['msl'].metpy.cartopy_crs
	ax_inset = fig.add_axes([0.125, 0.695, 0.2, 0.2], projection=data_crs)
	# Plot MSLP
	ax_inset.contour(mslp['longitude'], mslp['latitude'], mslp['msl'][i,:,:]/100, levels=mslp_levels, colors='blue', linewidths=0.5, transform=ccrs.PlateCarree())
	#ax_inset.scatter(start_liste,end_liste)
	#ax_inset.plot(cross['longitude'], cross['latitude'], transform=ccrs.PlateCarree(), color='black')
	# Plot the path of the cross section
	endpoints = data_crs.transform_points(ccrs.Geodetic(),*np.vstack([start_m, end_m]).transpose()[::-1])
	ax_inset.scatter(endpoints[:, 0], endpoints[:, 1], c='k', zorder=2)
	ax_inset.plot(cross_m['longitude'], cross_m['latitude'], c='k', zorder=2)
	# Set the titles and axes labels
	ax_inset.coastlines()
	ax_inset.gridlines()
	bounds = [(lonW, lonE, latS, latN)]
	ax_inset.set_extent(*bounds, crs=data_crs)
	#ax_inset.set_title('Coupe verticale et Pmer')

	#plt.show()
    
	figname='./figs/'+storm+'/Cross_section/PV_winds_meridional_'+str(time[i])[0:13]
	fig.savefig(figname+'.png',bbox_inches='tight')

	plt.close()

## Coupes SWNE

In [ ]:
for i in tqdm(range(len(time))):

	#print(str(time[i])[0:13])

	mslp2=mslp.sel(time=time[i])
	data2=data.sel(time=time[i])

#--  Cross sections with Metpy

# SWNE
	start_swne = (liste_lat[i]-8, liste_lon[i]-8)
	end_swne = (liste_lat[i]+8, liste_lon[i]+8)

	#print('Building cross sections ...')

	cross_swne=metpy.interpolate.cross_section(data2, start_swne, end_swne, steps=100, interp_type='linear').set_coords(('latitude', 'longitude'))		
	cross_swne['t_wind'], cross_swne['n_wind'] = mpcalc.cross_section_components(cross_swne['u_wind'],cross_swne['v_wind'])

	#print(cross_swne)


#--  plot SWNE cross section

	fig = plt.figure(figsize=(16., 9.))

	fig.suptitle('Storm '+storm+' - SWNE cross section  : Potential vorticity '+r"$\theta_e$ "+'and winds',fontsize=16)
	ax = plt.axes()
	ax.set_title(str(time[i])[0:13],loc='center',fontsize=14)
	ax.set_ylim(cross_swne['level'].max(), cross_swne['level'].min()) #met la pression min au sommet du graphe 
	ax.set_ylabel('Pressure (hPa)')
	ax.set_xlabel('Longitude')
	c = ax.contourf(cross_swne['longitude'], cross_swne['level'], cross_swne['pv']* 1e06, levels=pv_levels, cmap='YlOrRd',extend='both')	
	cb = fig.colorbar(c)
	cb.set_label('Potential vorticity', size='large')
	c1 = ax.contour(cross_swne['longitude'], cross_swne['level'], cross_swne['pv']* 1e06, levels = pv_levels, colors='grey', linewidths=0.0001)
	c2 = ax.contour(cross_swne['longitude'], cross_swne['level'], cross_swne['pv']* 1e06, levels=[1.5], colors='k', linewidths=2)
	c3 = ax.contour(cross_swne['longitude'], cross_z['level'], cross_z['thetae'], levels = th_levels, colors='brown', linewidths=1)
	ax.clabel(c3,fmt='%2.1i',fontsize=10)
	wind = ax.barbs(cross_swne['longitude'][5::6], cross_swne['level'], cross_swne['u_wind'][:,5::6], cross_swne['v_wind'][:,5::6], color='navy')

	# Define the CRS and inset axes
	data_crs = mslp['msl'].metpy.cartopy_crs
	ax_inset = fig.add_axes([0.125, 0.695, 0.2, 0.2], projection=data_crs)
	# Plot MSLP
	ax_inset.contour(mslp['longitude'], mslp['latitude'], mslp['msl'][i,:,:]/100, levels=mslp_levels, colors='blue', linewidths=0.5, transform=ccrs.PlateCarree())
	#ax_inset.scatter(start_liste,end_liste)
	#ax_inset.plot(cross['longitude'], cross['latitude'], transform=ccrs.PlateCarree(), color='black')
	# Plot the path of the cross section
	endpoints = data_crs.transform_points(ccrs.Geodetic(),*np.vstack([start_swne, end_swne]).transpose()[::-1])
	ax_inset.scatter(endpoints[:, 0], endpoints[:, 1], c='k', zorder=2)
	ax_inset.plot(cross_swne['longitude'], cross_swne['latitude'], c='k', zorder=2)
	# Set the titles and axes labels
	ax_inset.coastlines()
	ax_inset.gridlines()
	bounds = [(lonW, lonE, latS, latN)]
	ax_inset.set_extent(*bounds, crs=data_crs)
	#ax_inset.set_title('Coupe verticale et Pmer')

	#plt.show()
    
	figname='./figs/'+storm+'/Cross_section/PV_winds_SWNE_'+str(time[i])[0:13]
	fig.savefig(figname+'.png',bbox_inches='tight')

	plt.close()

## Coupes NWSE

In [ ]:
for i in tqdm(range(len(time))):

	#print(str(time[i])[0:13])

	mslp2=mslp.sel(time=time[i])
	data2=data.sel(time=time[i])

#--  Cross sections with Metpy

# nwse
	start_nwse= (liste_lat[i]+8, liste_lon[i]-8)
	end_nwse= (liste_lat[i]-8, liste_lon[i]+8)
    
	#print('Building cross sections ...')

	cross_nwse=metpy.interpolate.cross_section(data2, start_nwse, end_nwse, steps=100, interp_type='linear').set_coords(('latitude', 'longitude'))		
	cross_nwse['t_wind'], cross_nwse['n_wind'] = mpcalc.cross_section_components(cross_nwse['u_wind'],cross_nwse['v_wind'])
    
	#print(cross_swne)
	#print(cross_nwse)

#--  plot NWSE cross section

	fig = plt.figure(figsize=(16., 9.))

	fig.suptitle('Storm '+storm+' - NWSE cross section  : Potential vorticity '+r"$\theta_e$ "+'and winds',fontsize=16)
	ax = plt.axes()
	ax.set_title(str(time[i])[0:13],loc='center',fontsize=14)
	ax.set_ylim(cross_nwse['level'].max(), cross_nwse['level'].min()) #met la pression min au sommet du graphe 
	ax.set_ylabel('Pressure (hPa)')
	ax.set_xlabel('Longitude')
	c = ax.contourf(cross_nwse['longitude'], cross_nwse['level'], cross_nwse['pv']* 1e06, levels=pv_levels, cmap='YlOrRd',extend='both')	
	cb = fig.colorbar(c)
	cb.set_label('Potential vorticity', size='large')
	c1 = ax.contour(cross_nwse['longitude'], cross_nwse['level'], cross_nwse['pv']* 1e06, levels = pv_levels, colors='grey', linewidths=0.0001)
	c2 = ax.contour(cross_nwse['longitude'], cross_nwse['level'], cross_nwse['pv']* 1e06, levels=[1.5], colors='k', linewidths=2)
	c3 = ax.contour(cross_nwse['longitude'], cross_z['level'], cross_z['thetae'], levels = th_levels, colors='brown', linewidths=1)
	ax.clabel(c3,fmt='%2.1i',fontsize=10)
	wind = ax.barbs(cross_nwse['longitude'][5::6], cross_nwse['level'], cross_nwse['u_wind'][:,5::6], cross_nwse['v_wind'][:,5::6], color='navy')

	# Define the CRS and inset axes
	data_crs = mslp['msl'].metpy.cartopy_crs
	ax_inset = fig.add_axes([0.125, 0.695, 0.2, 0.2], projection=data_crs)
	# Plot MSLP
	ax_inset.contour(mslp['longitude'], mslp['latitude'], mslp['msl'][i,:,:]/100, levels=mslp_levels, colors='blue', linewidths=0.5, transform=ccrs.PlateCarree())
	#ax_inset.scatter(start_liste,end_liste)
	#ax_inset.plot(cross['longitude'], cross['latitude'], transform=ccrs.PlateCarree(), color='black')
	# Plot the path of the cross section
	endpoints = data_crs.transform_points(ccrs.Geodetic(),*np.vstack([start_nwse, end_nwse]).transpose()[::-1])
	ax_inset.scatter(endpoints[:, 0], endpoints[:, 1], c='k', zorder=2)
	ax_inset.plot(cross_nwse['longitude'], cross_nwse['latitude'], c='k', zorder=2)
	# Set the titles and axes labels
	ax_inset.coastlines()
	ax_inset.gridlines()
	bounds = [(lonW, lonE, latS, latN)]
	ax_inset.set_extent(*bounds, crs=data_crs)
	#ax_inset.set_title('Coupe verticale et Pmer')

	#plt.show()	

	figname='./figs/'+storm+'/Cross_section/PV_winds_NWSE_'+str(time[i])[0:13]
	fig.savefig(figname+'.png',bbox_inches='tight')

	plt.close()